# Compute statistics about NAVR maps

In [57]:
import pandas as pd
from pathlib import Path

anonymizer = True

root_dir = Path.cwd().parent.parent


def anondir(path: Path, prefix=root_dir) -> Path:
    """Anonymize a directory path by replacing user-specific parts with <root>."""
    if not anonymizer:
        return path
    path_str = str(path).replace(str(prefix), "<living-park>")
    return Path(path_str)


print(f"Root directory: {anondir(root_dir)}")
input_dir = root_dir / "npv" / "csv_all"
print(f"Input dir: {anondir(input_dir)}")
output_dir = root_dir / "npv" / "statistics"
output_dir.mkdir(parents=True, exist_ok=True)
print(f"Output dir: {anondir(output_dir)}")

Root directory: <living-park>
Input dir: <living-park>/npv/csv_all
Output dir: <living-park>/npv/statistics


In [58]:
subcortical_regions = [
    "Left-Thalamus",
    "Left-Caudate",
    "Left-Putamen",
    "Left-Pallidum",
    "Left-Hippocampus",
    "Left-Amygdala",
    "Left-Accumbens-area",
    "Right-Thalamus",
    "Right-Caudate",
    "Right-Putamen",
    "Right-Pallidum",
    "Right-Hippocampus",
    "Right-Amygdala",
    "Right-Accumbens-area",
]

# Cortical regions in DKT atlas
cortical_regions = [
    "bankssts",
    "caudalanteriorcingulate",
    "caudalmiddlefrontal",
    "cuneus",
    "entorhinal",
    "fusiform",
    "inferiorparietal",
    "inferiortemporal",
    "isthmuscingulate",
    "lateraloccipital",
    "lateralorbitofrontal",
    "lingual",
    "medialorbitofrontal",
    "middletemporal",
    "parahippocampal",
    "paracentral",
    "parsopercularis",
    "parsorbitalis",
    "parstriangularis",
    "pericalcarine",
    "postcentral",
    "posteriorcingulate",
    "precentral",
    "precuneus",
    "rostralanteriorcingulate",
    "rostralmiddlefrontal",
    "superiorfrontal",
    "superiorparietal",
    "superiortemporal",
    "supramarginal",
    "frontalpole",
    "temporalpole",
    "transversetemporal",
    "insula",
]

In [59]:
metrics = ["area", "thickness", "volume", "subcortical_volume"]
studies = ["cross-sectional", "longitudinal"]
timepoints = ["baseline", "followup"]
groups = ["HC", "PD"]


def is_cortical(metric):
    if metric not in metrics:
        raise ValueError(f"{metric} not in {metrics}")
    return metric != "subcortical_volume"


def _assert_args(metric, study, group, timepoint):
    if metric not in metrics:
        raise ValueError(f"{metric} not in {metrics}")
    if study not in studies:
        raise ValueError(f"{study} not in {studies}")
    if group not in groups:
        raise ValueError(f"{group} not in {groups}")
    if timepoint is not None and timepoint not in timepoints:
        raise ValueError(f"{timepoint} not in {timepoints}")


def get_stats(metric, study, group, timepoint=None):
    _assert_args(metric, study, group, timepoint)
    _study = study
    _timepoint = timepoint
    group = group.lower()
    if study == "cross-sectional":
        if timepoint is None:
            raise ValueError("Missing timepoint for cross-sectional")
        study = ""
        timepoint = f"_{timepoint}_"
    elif study == "longitudinal":
        study = "_" + study
        timepoint = "_"

    print(f"Statistics for {group} {_study} {_timepoint}")
    filename = input_dir / f"npv_{group}{timepoint}{metric}{study}.csv"
    df = pd.read_csv(filename)

    print(" - All regions")
    stats_all = df.describe()
    stats_all['group'] = group
    stats_all['timepoint'] = _timepoint
    stats_all['study'] = _study
    mean_all,std_all = stats_all['npv']['mean'], stats_all['npv']['std']
    print(f"\tnavr = {mean_all:.2f} ± {std_all:.2e}")
    
    regions = cortical_regions if is_cortical(metric) else subcortical_regions
    df = df[df['region'].isin(regions)]

    print(" - FreeSurfer regions")
    stats_fs = df.describe()
    stats_fs['group'] = group
    stats_fs['timepoint'] = _timepoint
    stats_fs['study'] = _study
    mean,std = stats_fs['npv']['mean'], stats_fs['npv']['std']
    print(f"\tnavr = {mean:.2f} ± {std:.2e}")

    return {'all':stats_all, 'freesurfer':stats_fs}

def get_stats_all(metric):
    df_all = {"all": pd.DataFrame(), "freesurfer": pd.DataFrame()}
    for study in studies:
        for group in groups:
            for timepoint in timepoints:
                if study == "longitudinal":
                    timepoint = None
                df = get_stats(metric, study, group, timepoint)
                df_all = {k: pd.concat([df_all[k], df[k]]) for k in df}
    return df_all

## Cortical area

In [60]:
df_area = get_stats_all('area')

Statistics for hc cross-sectional baseline
 - All regions
	navr = 0.16 ± 9.13e-02
 - FreeSurfer regions
	navr = 0.18 ± 8.45e-02
Statistics for hc cross-sectional followup
 - All regions
	navr = 0.15 ± 9.17e-02
 - FreeSurfer regions
	navr = 0.16 ± 8.73e-02
Statistics for pd cross-sectional baseline
 - All regions
	navr = 0.17 ± 9.14e-02
 - FreeSurfer regions
	navr = 0.18 ± 8.44e-02
Statistics for pd cross-sectional followup
 - All regions
	navr = 0.18 ± 9.32e-02
 - FreeSurfer regions
	navr = 0.19 ± 8.56e-02
Statistics for hc longitudinal None
 - All regions
	navr = 0.59 ± 1.81e-01
 - FreeSurfer regions
	navr = 0.62 ± 1.33e-01
Statistics for hc longitudinal None
 - All regions
	navr = 0.59 ± 1.81e-01
 - FreeSurfer regions
	navr = 0.62 ± 1.33e-01
Statistics for pd longitudinal None
 - All regions
	navr = 0.61 ± 1.75e-01
 - FreeSurfer regions
	navr = 0.65 ± 1.17e-01
Statistics for pd longitudinal None
 - All regions
	navr = 0.61 ± 1.75e-01
 - FreeSurfer regions
	navr = 0.65 ± 1.17e-01



## Cortical thickness

In [61]:
from pandas.core.frame import DataFrame

df_thickness: dict[str, DataFrame] = get_stats_all("thickness")

Statistics for hc cross-sectional baseline
 - All regions
	navr = 0.18 ± 7.09e-02
 - FreeSurfer regions
	navr = 0.19 ± 5.87e-02
Statistics for hc cross-sectional followup
 - All regions
	navr = 0.17 ± 6.97e-02
 - FreeSurfer regions
	navr = 0.19 ± 5.82e-02
Statistics for pd cross-sectional baseline
 - All regions
	navr = 0.20 ± 8.02e-02
 - FreeSurfer regions
	navr = 0.22 ± 6.63e-02
Statistics for pd cross-sectional followup
 - All regions
	navr = 0.20 ± 8.46e-02
 - FreeSurfer regions
	navr = 0.22 ± 7.23e-02
Statistics for hc longitudinal None
 - All regions
	navr = 0.43 ± 1.27e-01
 - FreeSurfer regions
	navr = 0.45 ± 1.00e-01
Statistics for hc longitudinal None
 - All regions
	navr = 0.43 ± 1.27e-01
 - FreeSurfer regions
	navr = 0.45 ± 1.00e-01
Statistics for pd longitudinal None
 - All regions
	navr = 0.43 ± 1.32e-01
 - FreeSurfer regions
	navr = 0.46 ± 1.06e-01
Statistics for pd longitudinal None
 - All regions
	navr = 0.43 ± 1.32e-01
 - FreeSurfer regions
	navr = 0.46 ± 1.06e-01


## Cortical volume

In [62]:
df_volume = get_stats_all("volume")

Statistics for hc cross-sectional baseline
 - All regions
	navr = 0.15 ± 7.84e-02
 - FreeSurfer regions
	navr = 0.16 ± 7.26e-02
Statistics for hc cross-sectional followup
 - All regions
	navr = 0.14 ± 7.96e-02
 - FreeSurfer regions
	navr = 0.15 ± 7.55e-02
Statistics for pd cross-sectional baseline
 - All regions
	navr = 0.17 ± 9.22e-02
 - FreeSurfer regions
	navr = 0.17 ± 8.69e-02
Statistics for pd cross-sectional followup
 - All regions
	navr = 0.17 ± 9.28e-02
 - FreeSurfer regions
	navr = 0.18 ± 8.69e-02
Statistics for hc longitudinal None
 - All regions
	navr = 0.54 ± 1.59e-01
 - FreeSurfer regions
	navr = 0.56 ± 1.22e-01
Statistics for hc longitudinal None
 - All regions
	navr = 0.54 ± 1.59e-01
 - FreeSurfer regions
	navr = 0.56 ± 1.22e-01
Statistics for pd longitudinal None
 - All regions
	navr = 0.56 ± 1.70e-01
 - FreeSurfer regions
	navr = 0.59 ± 1.32e-01
Statistics for pd longitudinal None
 - All regions
	navr = 0.56 ± 1.70e-01
 - FreeSurfer regions
	navr = 0.59 ± 1.32e-01


## Subcortical volume

In [63]:
df_subcortical_volume = get_stats_all("subcortical_volume")

Statistics for hc cross-sectional baseline
 - All regions
	navr = 0.16 ± 1.64e-01
 - FreeSurfer regions
	navr = 0.18 ± 5.51e-02
Statistics for hc cross-sectional followup
 - All regions
	navr = 0.17 ± 1.64e-01
 - FreeSurfer regions
	navr = 0.18 ± 4.70e-02
Statistics for pd cross-sectional baseline
 - All regions
	navr = 0.18 ± 1.86e-01
 - FreeSurfer regions
	navr = 0.19 ± 5.64e-02
Statistics for pd cross-sectional followup
 - All regions
	navr = 0.17 ± 1.77e-01
 - FreeSurfer regions
	navr = 0.18 ± 5.42e-02
Statistics for hc longitudinal None
 - All regions
	navr = 0.48 ± 2.31e-01
 - FreeSurfer regions
	navr = 0.59 ± 5.54e-02
Statistics for hc longitudinal None
 - All regions
	navr = 0.48 ± 2.31e-01
 - FreeSurfer regions
	navr = 0.59 ± 5.54e-02
Statistics for pd longitudinal None
 - All regions
	navr = 0.46 ± 2.04e-01
 - FreeSurfer regions
	navr = 0.54 ± 7.35e-02
Statistics for pd longitudinal None
 - All regions
	navr = 0.46 ± 2.04e-01
 - FreeSurfer regions
	navr = 0.54 ± 7.35e-02


### all metrics

In [64]:
df_global_all = pd.DataFrame()
df_global_fs = pd.DataFrame()
for group in groups:
    for study in studies:
        for timepoint in timepoints:
            if study == "longitudinal":
                timepoint = None
                
            df = pd.DataFrame()
            for metric, df_metric in zip(
                ["area", "thickness", "volume", "subcortical_volume"],
                [df_area, df_thickness, df_volume, df_subcortical_volume],
            ):
                group = group.lower()
                if study == "cross-sectional":
                    _study = ""
                    _timepoint = f"_{timepoint}_"
                elif study == "longitudinal":
                    _study = "_" + study
                    _timepoint = "_"
                else:
                    msg = f"Unknown study: {study}"
                    raise ValueError(msg)
                filename = input_dir / f"npv_{group}{_timepoint}{metric}{_study}.csv"
                df = pd.concat([df, pd.read_csv(filename)])

            df_stat_all = df.describe()
            df_stat_all['group'] = group
            df_stat_all['timepoint'] = timepoint
            df_stat_all['study'] = study

            df_stat_fs = df[df['region'].isin(cortical_regions + subcortical_regions)].describe()
            df_stat_fs['group'] = group
            df_stat_fs['timepoint'] = timepoint
            df_stat_fs['study'] = study
            
            print(f"Statistics for {group} {study} {timepoint} - All metrics")
            mean,std = df_stat_all['npv']['mean'], df_stat_all['npv']['std']
            print(f"\tnavr = {mean:.2f} ± {std:.2e}")

            df_global_all = pd.concat([df_global_all, df_stat_all])
            df_global_fs = pd.concat([df_global_fs, df_stat_fs])
                
df_global = {"all": df_global_all, "freesurfer": df_global_fs}

Statistics for hc cross-sectional baseline - All metrics
	navr = 0.16 ± 1.04e-01
Statistics for hc cross-sectional followup - All metrics
	navr = 0.16 ± 1.05e-01
Statistics for hc longitudinal None - All metrics
	navr = 0.51 ± 1.85e-01
Statistics for hc longitudinal None - All metrics
	navr = 0.51 ± 1.85e-01
Statistics for pd cross-sectional baseline - All metrics
	navr = 0.18 ± 1.17e-01
Statistics for pd cross-sectional followup - All metrics
	navr = 0.18 ± 1.15e-01
Statistics for pd longitudinal None - All metrics
	navr = 0.52 ± 1.84e-01
Statistics for pd longitudinal None - All metrics
	navr = 0.52 ± 1.84e-01


## Table

In [65]:
from rich.console import Console
from rich.table import Table
from rich.text import Text
from rich import box
import numpy as np
import pandas as pd

# --- tiny helpers -------------------------------------------------------------


def _clip01(x: float) -> float:
    return float(min(1.0, max(0.0, x)))


def _npv_color(x: float) -> str:
    """
    Green→Yellow→Red gradient for npv in [0,1].
    """
    x = _clip01(x)
    if x <= 0.5:
        r = int(510 * x)  # 0→255
        g = 255
    else:
        r = 255
        g = int(510 * (1 - x))  # 255→0
    return f"rgb({r},{g},0)"


def _bar(x: float, width: int = 10) -> Text:
    """Simple 10-block bar for npv."""
    x = _clip01(x)
    n_full = int(round(x * width))
    t = Text()
    t.append("█" * n_full, style=_npv_color(x))
    t.append("░" * (width - n_full), style="grey46")
    return t


# --- main pretty table --------------------------------------------------------


def show_navr_table(df: pd.DataFrame, title: str = "Dataset"):
    """
    Prints a compact, colorful Rich table.
    - If a column name contains 'npv' (case-insensitive): color + bar.
    - Other numeric columns: light coloring by magnitude.
    - Works with a normal or MultiIndex index.
    """
    console = Console()
    tbl = Table(
        title=f"📊 {title}",
        box=box.SIMPLE_HEAVY,
        border_style="grey54",
        header_style="bold white on dark_blue",
        show_lines=False,
        pad_edge=False,
        expand=True,
        row_styles=["none", "dim"],
    )

    # Index column(s)
    if isinstance(df.index, pd.MultiIndex):
        for i, name in enumerate(df.index.names):
            tbl.add_column(name or f"Index_{i}", style="bold cyan", no_wrap=True)
    else:
        tbl.add_column(df.index.name or "Index", style="bold cyan", no_wrap=True)

    # Data columns
    for col in df.columns:
        tbl.add_column(str(col), justify="right")

    # Add rows
    for idx, row in df.iterrows():
        cells = []

        # index cells
        if isinstance(df.index, pd.MultiIndex):
            for v in idx if isinstance(idx, tuple) else (idx,):
                cells.append(Text(str(v), style="bold cyan"))
        else:
            cells.append(Text(str(idx), style="bold cyan"))

        # value cells
        for col, val in row.items():
            if isinstance(val, (int, float, np.floating)):
                col_upper = str(col).upper()
                if "NPV" in col_upper:
                    txt = Text(f"{val:0.3f} ", style=_npv_color(val))
                    bar = _bar(val)                 # bar is already a Text()
                    txt.append(bar)                 # ✅ no style argument when appending Text
                    cells.append(txt)
                else:
                    style = "bright_white" if abs(val) > 1 else "white"
                    cells.append(Text(f"{val:0.3f}", style=style))

            else:
                cells.append(Text(str(val)))

        tbl.add_row(*cells)

    console.print(tbl)

    # tiny legend
    console.print(
        Text("npv:", style="bold")
        + Text(" low ", style=_npv_color(0.05))
        + Text("▸", style="white")
        + Text(" mid ", style=_npv_color(0.5))
        + Text("▸", style="white")
        + Text(" high", style=_npv_color(0.95)),
    )

def show_navr_stats(df, region, metric):
    df = df[region].sort_index().reset_index().rename(columns={'index':'statistics'})
    df = df[df['statistics'].isin(['mean','std','max'])].drop(columns=['n']).set_index('statistics')
    show_navr_table(df.sort_index(), metric)

### All metrics

In [66]:
show_navr_stats(df_global, "all", "All metrics")

                                                  📊 All metrics                                                   
                                                                                                                   
 statistics            npv        num         anat    npv_CI_low    npv_CI_high   group   timepoint          study 
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
 max                 0.985   2785.236   161014.082         0.945          1.026      pd    baseline   cross-secti… 
                ██████████                            █████████░     ██████████                                    
 max                 0.950      2.967        1.263         0.923          0.978      pd        None   longitudinal 
                ██████████                            █████████░     ██████████                                    
 max                 0.913   3140.333   159322.753         0.877          0.952      pd    followup   cross-secti… 
                █████████░                            █████████░     ██████████                                    
 max                 0.908      0.889        1.253         0.884          0.932      hc        None   longitudinal 
                █████████░                            █████████░     █████████░                                    
 max                 0.950      2.967        1.263         0.923          0.978      pd        None   longitudinal 
                ██████████                            █████████░     ██████████                                    
 max                 0.874   2720.287   169961.239         0.842          0.908      hc    baseline   cross-secti… 
                █████████░                            ████████░░     █████████░                                    
 max                 0.845   2726.654   164448.190         0.813          0.878      hc    followup   cross-secti… 
                ████████░░                            ████████░░     █████████░                                    
 max                 0.908      0.889        1.253         0.884          0.932      hc        None   longitudinal 
                █████████░                            █████████░     █████████░                                    
 mean                0.164    198.145     9983.677         0.158          0.170      hc    baseline   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.518      0.054        0.073         0.503          0.533      pd        None   longitudinal 
                █████░░░░░                            █████░░░░░     █████░░░░░                                    
 mean                0.157    194.740    10120.815         0.151          0.163      hc    followup   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.509      0.039        0.066         0.496          0.523      hc        None   longitudinal 
                █████░░░░░                            █████░░░░░     █████░░░░░                                    
 mean                0.518      0.054        0.073         0.503          0.533      pd        None   longitudinal 
                █████░░░░░                            █████░░░░░     █████░░░░░                                    
 mean                0.509      0.039        0.066         0.496          0.523      hc        None   longitudinal 
                █████░░░░░                            █████░░░░░     █████░░░░░                                    
 mean                0.179    197.437     9185.640         0.171          0.186      pd    baseline   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.181    214.702     9092.005       

npv: low ▸ mid ▸ high

In [67]:
show_navr_stats(df_global, "freesurfer", "All metrics")

                                                  📊 All metrics                                                   
                                                                                                                   
 statistics            npv       num       anat     npv_CI_low    npv_CI_high   group   timepoint            study 
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
 max                 0.455   380.676   2543.247          0.436          0.474      pd    baseline   cross-section… 
                █████░░░░░                          ████░░░░░░     █████░░░░░                                      
 max                 0.891     0.121      0.160          0.865          0.917      pd        None     longitudinal 
                █████████░                          █████████░     █████████░                                      
 max                 0.461   431.147   2504.848          0.443          0.481      pd    followup   cross-section… 
                █████░░░░░                          ████░░░░░░     █████░░░░░                                      
 max                 0.895     0.147      0.252          0.872          0.920      hc        None     longitudinal 
                █████████░                          █████████░     █████████░                                      
 max                 0.891     0.121      0.160          0.865          0.917      pd        None     longitudinal 
                █████████░                          █████████░     █████████░                                      
 max                 0.424   492.738   2647.919          0.408          0.440      hc    baseline   cross-section… 
                ████░░░░░░                          ████░░░░░░     ████░░░░░░                                      
 max                 0.402   250.703   2689.984          0.387          0.418      hc    followup   cross-section… 
                ████░░░░░░                          ████░░░░░░     ████░░░░░░                                      
 max                 0.895     0.147      0.252          0.872          0.920      hc        None     longitudinal 
                █████████░                          █████████░     █████████░                                      
 mean                0.176    63.168    443.323          0.169          0.183      hc    baseline   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.561     0.032      0.055          0.545          0.578      pd        None     longitudinal 
                ██████░░░░                          █████░░░░░     ██████░░░░                                      
 mean                0.165    54.192    451.995          0.158          0.171      hc    followup   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.549     0.028      0.049          0.534          0.564      hc        None     longitudinal 
                █████░░░░░                          █████░░░░░     ██████░░░░                                      
 mean                0.561     0.032      0.055          0.545          0.578      pd        None     longitudinal 
                ██████░░░░                          █████░░░░░     ██████░░░░                                      
 mean                0.549     0.028      0.049          0.534          0.564      hc        None     longitudinal 
                █████░░░░░                          █████░░░░░     ██████░░░░                                      
 mean                0.191    60.156    405.437          0.183          0.199      pd    baseline   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.195    63.340    403.494          

npv: low ▸ mid ▸ high

### Cortical Area

In [68]:
show_navr_stats(df_area, "all", "Cortical Surface Area")

                                             📊 Cortical Surface Area                                              
                                                                                                                   
 statistics            npv        num         anat    npv_CI_low    npv_CI_high   group   timepoint          study 
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
 max                 0.895      0.090        0.110         0.872          0.920      hc        None   longitudinal 
                █████████░                            █████████░     █████████░                                    
 max                 0.891      0.099        0.135         0.865          0.917      pd        None   longitudinal 
                █████████░                            █████████░     █████████░                                    
 max                 0.895      0.090        0.110         0.872          0.920      hc        None   longitudinal 
                █████████░                            █████████░     █████████░                                    
 max                 0.435   3019.072   159322.753         0.418          0.454      pd    followup   cross-secti… 
                ████░░░░░░                            ████░░░░░░     █████░░░░░                                    
 max                 0.891      0.099        0.135         0.865          0.917      pd        None   longitudinal 
                █████████░                            █████████░     █████████░                                    
 max                 0.424   2720.287   169961.239         0.408          0.440      hc    baseline   cross-secti… 
                ████░░░░░░                            ████░░░░░░     ████░░░░░░                                    
 max                 0.402   2726.654   164448.190         0.387          0.418      hc    followup   cross-secti… 
                ████░░░░░░                            ████░░░░░░     ████░░░░░░                                    
 max                 0.421   2782.518   161014.082         0.404          0.439      pd    baseline   cross-secti… 
                ████░░░░░░                            ████░░░░░░     ████░░░░░░                                    
 mean                0.163    134.058     8111.951         0.157          0.169      hc    baseline   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.607      0.035        0.054         0.590          0.625      pd        None   longitudinal 
                ██████░░░░                            ██████░░░░     ██████░░░░                                    
 mean                0.148    126.153     8124.133         0.142          0.154      hc    followup   cross-secti… 
                █░░░░░░░░░                            █░░░░░░░░░     ██░░░░░░░░                                    
 mean                0.168    133.069     7475.329         0.161          0.175      pd    baseline   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.607      0.035        0.054         0.590          0.625      pd        None   longitudinal 
                ██████░░░░                            ██████░░░░     ██████░░░░                                    
 mean                0.177    143.732     7378.890         0.169          0.184      pd    followup   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.587      0.030        0.048         0.571          0.603      hc        None   longitudinal 
                ██████░░░░                            ██████░░░░     ██████░░░░                                    
 mean                0.587      0.030        0.048       

npv: low ▸ mid ▸ high

In [69]:
show_navr_stats(df_area, "freesurfer", "Cortical Surface Area")

                                             📊 Cortical Surface Area                                              
                                                                                                                   
 statistics            npv       num      anat      npv_CI_low    npv_CI_high   group   timepoint            study 
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
 max                 0.895     0.090     0.110           0.872          0.920      hc        None     longitudinal 
                █████████░                          █████████░     █████████░                                      
 max                 0.891     0.099     0.135           0.865          0.917      pd        None     longitudinal 
                █████████░                          █████████░     █████████░                                      
 max                 0.895     0.090     0.110           0.872          0.920      hc        None     longitudinal 
                █████████░                          █████████░     █████████░                                      
 max                 0.435   170.676   894.196           0.418          0.454      pd    followup   cross-section… 
                ████░░░░░░                          ████░░░░░░     █████░░░░░                                      
 max                 0.891     0.099     0.135           0.865          0.917      pd        None     longitudinal 
                █████████░                          █████████░     █████████░                                      
 max                 0.424   180.076   960.187           0.408          0.440      hc    baseline   cross-section… 
                ████░░░░░░                          ████░░░░░░     ████░░░░░░                                      
 max                 0.402   114.335   982.661           0.387          0.418      hc    followup   cross-section… 
                ████░░░░░░                          ████░░░░░░     ████░░░░░░                                      
 max                 0.421   144.623   940.456           0.404          0.439      pd    baseline   cross-section… 
                ████░░░░░░                          ████░░░░░░     ████░░░░░░                                      
 mean                0.175    51.208   349.631           0.169          0.182      hc    baseline   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.645     0.038     0.057           0.627          0.664      pd        None     longitudinal 
                ██████░░░░                          ██████░░░░     ███████░░░                                      
 mean                0.159    43.112   357.205           0.153          0.165      hc    followup   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.181    48.479   326.045           0.173          0.188      pd    baseline   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.645     0.038     0.057           0.627          0.664      pd        None     longitudinal 
                ██████░░░░                          ██████░░░░     ███████░░░                                      
 mean                0.190    51.294   321.366           0.182          0.198      pd    followup   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.623     0.032     0.050           0.607          0.640      hc        None     longitudinal 
                ██████░░░░                          ██████░░░░     ██████░░░░                                      
 mean                0.623     0.032     0.050           

npv: low ▸ mid ▸ high

### Cortical Thickness

In [70]:
show_navr_stats(df_thickness, "all", "Cortical Thickness")

                                               📊 Cortical Thickness                                               
                                                                                                                   
 statistics            npv        num         anat    npv_CI_low    npv_CI_high   group   timepoint          study 
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
 max                 0.732      0.041        0.087         0.712          0.751      hc        None   longitudinal 
                ███████░░░                            ███████░░░     ████████░░                                    
 max                 0.695      0.050        0.104         0.675          0.716      pd        None   longitudinal 
                ███████░░░                            ███████░░░     ███████░░░                                    
 max                 0.732      0.041        0.087         0.712          0.751      hc        None   longitudinal 
                ███████░░░                            ███████░░░     ████████░░                                    
 max                 0.390   3019.072   159322.753         0.374          0.406      pd    followup   cross-secti… 
                ████░░░░░░                            ████░░░░░░     ████░░░░░░                                    
 max                 0.695      0.050        0.104         0.675          0.716      pd        None   longitudinal 
                ███████░░░                            ███████░░░     ███████░░░                                    
 max                 0.328   2720.287   169961.239         0.316          0.341      hc    baseline   cross-secti… 
                ███░░░░░░░                            ███░░░░░░░     ███░░░░░░░                                    
 max                 0.348   2726.654   164448.190         0.335          0.362      hc    followup   cross-secti… 
                ███░░░░░░░                            ███░░░░░░░     ████░░░░░░                                    
 max                 0.451   2782.518   161014.082         0.433          0.470      pd    baseline   cross-secti… 
                █████░░░░░                            ████░░░░░░     █████░░░░░                                    
 mean                0.178     77.264     7563.366         0.171          0.185      hc    baseline   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.435      0.017        0.038         0.422          0.448      pd        None   longitudinal 
                ████░░░░░░                            ████░░░░░░     ████░░░░░░                                    
 mean                0.174     75.772     7557.367         0.168          0.181      hc    followup   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.204     77.358     6964.388         0.196          0.213      pd    baseline   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.435      0.017        0.038         0.422          0.448      pd        None   longitudinal 
                ████░░░░░░                            ████░░░░░░     ████░░░░░░                                    
 mean                0.203     84.866     6876.436         0.195          0.211      pd    followup   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.433      0.015        0.034         0.421          0.444      hc        None   longitudinal 
                ████░░░░░░                            ████░░░░░░     ████░░░░░░                                    
 mean                0.433      0.015        0.034       

npv: low ▸ mid ▸ high

In [71]:
show_navr_stats(df_thickness, "freesurfer", "Cortical Thickness")

                                               📊 Cortical Thickness                                               
                                                                                                                   
 statistics              npv     num    anat      npv_CI_low      npv_CI_high   group   timepoint            study 
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
 max                   0.732   0.041   0.087           0.712            0.751      hc        None     longitudinal 
                  ███████░░░                      ███████░░░       ████████░░                                      
 max                   0.695   0.050   0.104           0.675            0.716      pd        None     longitudinal 
                  ███████░░░                      ███████░░░       ███████░░░                                      
 max                   0.732   0.041   0.087           0.712            0.751      hc        None     longitudinal 
                  ███████░░░                      ███████░░░       ████████░░                                      
 max                   0.390   0.102   0.283           0.374            0.406      pd    followup   cross-section… 
                  ████░░░░░░                      ████░░░░░░       ████░░░░░░                                      
 max                   0.695   0.050   0.104           0.675            0.716      pd        None     longitudinal 
                  ███████░░░                      ███████░░░       ███████░░░                                      
 max                   0.328   0.088   0.293           0.316            0.341      hc    baseline   cross-section… 
                  ███░░░░░░░                      ███░░░░░░░       ███░░░░░░░                                      
 max                   0.348   0.095   0.296           0.335            0.362      hc    followup   cross-section… 
                  ███░░░░░░░                      ███░░░░░░░       ████░░░░░░                                      
 max                   0.451   0.118   0.334           0.433            0.470      pd    baseline   cross-section… 
                  █████░░░░░                      ████░░░░░░       █████░░░░░                                      
 mean                  0.190   0.029   0.143           0.183            0.198      hc    baseline   cross-section… 
                  ██░░░░░░░░                      ██░░░░░░░░       ██░░░░░░░░                                      
 mean                  0.457   0.019   0.040           0.444            0.471      pd        None     longitudinal 
                  █████░░░░░                      ████░░░░░░       █████░░░░░                                      
 mean                  0.186   0.029   0.147           0.179            0.193      hc    followup   cross-section… 
                  ██░░░░░░░░                      ██░░░░░░░░       ██░░░░░░░░                                      
 mean                  0.218   0.032   0.140           0.210            0.228      pd    baseline   cross-section… 
                  ██░░░░░░░░                      ██░░░░░░░░       ██░░░░░░░░                                      
 mean                  0.457   0.019   0.040           0.444            0.471      pd        None     longitudinal 
                  █████░░░░░                      ████░░░░░░       █████░░░░░                                      
 mean                  0.217   0.032   0.140           0.208            0.226      pd    followup   cross-section… 
                  ██░░░░░░░░                      ██░░░░░░░░       ██░░░░░░░░                                      
 mean                  0.454   0.016   0.035           0.442            0.467      hc        None     longitudinal 
                  █████░░░░░                      ████░░░░░░       █████░░░░░                                      
 mean                  0.454   0.016   0.035           0.

npv: low ▸ mid ▸ high

### Cortical Volume

In [72]:
show_navr_stats(df_volume, "all", "Cortical Volume")

                                                📊 Cortical Volume                                                 
                                                                                                                   
 statistics            npv        num         anat    npv_CI_low    npv_CI_high   group   timepoint          study 
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
 max                 0.826      0.147        0.252         0.805          0.849      hc        None   longitudinal 
                ████████░░                            ████████░░     ████████░░                                    
 max                 0.818      0.121        0.160         0.794          0.842      pd        None   longitudinal 
                ████████░░                            ████████░░     ████████░░                                    
 max                 0.826      0.147        0.252         0.805          0.849      hc        None   longitudinal 
                ████████░░                            ████████░░     ████████░░                                    
 max                 0.461   3019.072   159322.753         0.443          0.481      pd    followup   cross-secti… 
                █████░░░░░                            ████░░░░░░     █████░░░░░                                    
 max                 0.818      0.121        0.160         0.794          0.842      pd        None   longitudinal 
                ████████░░                            ████████░░     ████████░░                                    
 max                 0.383   2720.287   169961.239         0.369          0.398      hc    baseline   cross-secti… 
                ████░░░░░░                            ████░░░░░░     ████░░░░░░                                    
 max                 0.373   2726.654   164448.190         0.359          0.388      hc    followup   cross-secti… 
                ████░░░░░░                            ████░░░░░░     ████░░░░░░                                    
 max                 0.455   2782.518   161014.082         0.436          0.474      pd    baseline   cross-secti… 
                █████░░░░░                            ████░░░░░░     █████░░░░░                                    
 mean                0.153    209.199     8703.578         0.147          0.159      hc    baseline   cross-secti… 
                ██░░░░░░░░                            █░░░░░░░░░     ██░░░░░░░░                                    
 mean                0.561      0.037        0.062         0.545          0.578      pd        None   longitudinal 
                ██████░░░░                            █████░░░░░     ██████░░░░                                    
 mean                0.138    187.428     8715.041         0.133          0.144      hc    followup   cross-secti… 
                █░░░░░░░░░                            █░░░░░░░░░     █░░░░░░░░░                                    
 mean                0.166    203.076     8002.528         0.159          0.173      pd    baseline   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.561      0.037        0.062         0.545          0.578      pd        None   longitudinal 
                ██████░░░░                            █████░░░░░     ██████░░░░                                    
 mean                0.173    218.531     7909.293         0.166          0.180      pd    followup   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.536      0.032        0.056         0.522          0.551      hc        None   longitudinal 
                █████░░░░░                            █████░░░░░     ██████░░░░                                    
 mean                0.536      0.032        0.056       

npv: low ▸ mid ▸ high

In [73]:
show_navr_stats(df_volume, "freesurfer", "Cortical Volume")

                                                📊 Cortical Volume                                                 
                                                                                                                   
 statistics            npv       num       anat     npv_CI_low    npv_CI_high   group   timepoint            study 
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
 max                 0.826     0.147      0.252          0.805          0.849      hc        None     longitudinal 
                ████████░░                          ████████░░     ████████░░                                      
 max                 0.818     0.121      0.160          0.794          0.842      pd        None     longitudinal 
                ████████░░                          ████████░░     ████████░░                                      
 max                 0.826     0.147      0.252          0.805          0.849      hc        None     longitudinal 
                ████████░░                          ████████░░     ████████░░                                      
 max                 0.461   431.147   2504.848          0.443          0.481      pd    followup   cross-section… 
                █████░░░░░                          ████░░░░░░     █████░░░░░                                      
 max                 0.818     0.121      0.160          0.794          0.842      pd        None     longitudinal 
                ████████░░                          ████████░░     ████████░░                                      
 max                 0.383   492.738   2647.919          0.369          0.398      hc    baseline   cross-section… 
                ████░░░░░░                          ████░░░░░░     ████░░░░░░                                      
 max                 0.373   250.703   2689.984          0.359          0.388      hc    followup   cross-section… 
                ████░░░░░░                          ████░░░░░░     ████░░░░░░                                      
 max                 0.455   380.676   2543.247          0.436          0.474      pd    baseline   cross-section… 
                █████░░░░░                          ████░░░░░░     █████░░░░░                                      
 mean                0.161   137.453    984.977          0.155          0.167      hc    baseline   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.587     0.039      0.065          0.570          0.604      pd        None     longitudinal 
                ██████░░░░                          ██████░░░░     ██████░░░░                                      
 mean                0.146   116.025   1003.647          0.140          0.152      hc    followup   cross-section… 
                █░░░░░░░░░                          █░░░░░░░░░     ██░░░░░░░░                                      
 mean                0.175   130.870    894.514          0.168          0.182      pd    baseline   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.587     0.039      0.065          0.570          0.604      pd        None     longitudinal 
                ██████░░░░                          ██████░░░░     ██████░░░░                                      
 mean                0.182   139.065    891.507          0.175          0.190      pd    followup   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.561     0.034      0.058          0.547          0.576      hc        None     longitudinal 
                ██████░░░░                          █████░░░░░     ██████░░░░                                      
 mean                0.561     0.034      0.058          

npv: low ▸ mid ▸ high

### Subcortical Volume

In [74]:
show_navr_stats(df_subcortical_volume, 'all', "Subcortical volume")

                                               📊 Subcortical volume                                               
                                                                                                                   
 statistics            npv        num         anat    npv_CI_low    npv_CI_high   group   timepoint          study 
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
 max                 0.908      0.889        1.253         0.884          0.932      hc        None   longitudinal 
                █████████░                            █████████░     █████████░                                    
 max                 0.950      2.967        1.263         0.923          0.978      pd        None   longitudinal 
                ██████████                            █████████░     ██████████                                    
 max                 0.908      0.889        1.253         0.884          0.932      hc        None   longitudinal 
                █████████░                            █████████░     █████████░                                    
 max                 0.913   3140.333   159322.753         0.877          0.952      pd    followup   cross-secti… 
                █████████░                            █████████░     ██████████                                    
 max                 0.950      2.967        1.263         0.923          0.978      pd        None   longitudinal 
                ██████████                            █████████░     ██████████                                    
 max                 0.874   2720.287   169961.239         0.842          0.908      hc    baseline   cross-secti… 
                █████████░                            ████████░░     █████████░                                    
 max                 0.845   2726.654   164448.190         0.813          0.878      hc    followup   cross-secti… 
                ████████░░                            ████████░░     █████████░                                    
 max                 0.985   2785.236   161014.082         0.945          1.026      pd    baseline   cross-secti… 
                ██████████                            █████████░     ██████████                                    
 mean                0.161    396.481    16287.953         0.155          0.167      hc    baseline   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.457      0.143        0.155         0.444          0.471      pd        None   longitudinal 
                █████░░░░░                            ████░░░░░░     █████░░░░░                                    
 mean                0.169    416.366    16869.512         0.162          0.175      hc    followup   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.175    401.176    14972.097         0.168          0.183      pd    baseline   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.457      0.143        0.155         0.444          0.471      pd        None   longitudinal 
                █████░░░░░                            ████░░░░░░     █████░░░░░                                    
 mean                0.170    439.072    14874.741         0.163          0.177      pd    followup   cross-secti… 
                ██░░░░░░░░                            ██░░░░░░░░     ██░░░░░░░░                                    
 mean                0.476      0.087        0.141         0.463          0.489      hc        None   longitudinal 
                █████░░░░░                            █████░░░░░     █████░░░░░                                    
 mean                0.476      0.087        0.141       

npv: low ▸ mid ▸ high

In [75]:
show_navr_stats(df_subcortical_volume, 'freesurfer', "Subcortical volume")

                                               📊 Subcortical volume                                               
                                                                                                                   
 statistics            npv       num      anat      npv_CI_low    npv_CI_high   group   timepoint            study 
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
 max                 0.666     0.084     0.145           0.649          0.684      hc        None     longitudinal 
                ███████░░░                          ██████░░░░     ███████░░░                                      
 max                 0.685     0.077     0.154           0.665          0.705      pd        None     longitudinal 
                ███████░░░                          ███████░░░     ███████░░░                                      
 max                 0.666     0.084     0.145           0.649          0.684      hc        None     longitudinal 
                ███████░░░                          ██████░░░░     ███████░░░                                      
 max                 0.274   123.764   773.087           0.263          0.286      pd    followup   cross-section… 
                ███░░░░░░░                          ███░░░░░░░     ███░░░░░░░                                      
 max                 0.685     0.077     0.154           0.665          0.705      pd        None     longitudinal 
                ███████░░░                          ███████░░░     ███████░░░                                      
 max                 0.280   151.460   927.673           0.270          0.291      hc    baseline   cross-section… 
                ███░░░░░░░                          ███░░░░░░░     ███░░░░░░░                                      
 max                 0.268   157.152   909.344           0.258          0.279      hc    followup   cross-section… 
                ███░░░░░░░                          ███░░░░░░░     ███░░░░░░░                                      
 max                 0.292   133.404   813.834           0.280          0.304      pd    baseline   cross-section… 
                ███░░░░░░░                          ███░░░░░░░     ███░░░░░░░                                      
 mean                0.180    67.125   420.097           0.174          0.187      hc    baseline   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.536     0.035     0.066           0.520          0.551      pd        None     longitudinal 
                █████░░░░░                          █████░░░░░     ██████░░░░                                      
 mean                0.179    70.762   427.637           0.172          0.186      hc    followup   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.191    65.441   384.123           0.183          0.199      pd    baseline   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.536     0.035     0.066           0.520          0.551      pd        None     longitudinal 
                █████░░░░░                          █████░░░░░     ██████░░░░                                      
 mean                0.177    61.542   391.192           0.170          0.184      pd    followup   cross-section… 
                ██░░░░░░░░                          ██░░░░░░░░     ██░░░░░░░░                                      
 mean                0.586     0.036     0.061           0.571          0.602      hc        None     longitudinal 
                ██████░░░░                          ██████░░░░     ██████░░░░                                      
 mean                0.586     0.036     0.061           

npv: low ▸ mid ▸ high

## Boostrap test to compare mean navr between PD and HC

Group-level differences in $\nu_{nav}$ between healthy controls (HC) and patients with Parkinson's disease (PD) is evaluated using a non-parametric bootstrap test (10,000 iterations). 

For each iteration, regions were sampled with replacement, and the mean $\nu_{nav}$
difference (PD - HC) is recomputed to estimate the empirical null distribution. 

Two-sided p-values is computed as the proportion of bootstrap differences with absolute value greater than or equal to the observed difference.


In [99]:
import pandas as pd
import numpy as np
from scipy.stats import bootstrap, permutation_test

alpha = 0.05

def bootstrap_group_diff(df_hc, df_pd, value_col="npv", n_boot=10000, seed=0, ci=0.95):
    """
    Compare mean NAVR between HC and PD using scipy.stats.bootstrap.

    Automatically handles metrics with or without 'hemisphere' column.

    Parameters
    ----------
    df_hc, df_pd : pandas.DataFrame
        DataFrames with at least ['region', value_col] and optionally ['hemisphere'].
    value_col : str
        Column containing NAVR values.
    n_boot : int
        Number of bootstrap resamples.
    seed : int
        Random seed for reproducibility.
    ci : float
        Confidence interval level.

    Returns
    -------
    observed : float
        Observed mean difference (PD–HC).
    ci_interval : tuple
        Confidence interval for the mean difference.
    p_value : float
        Two-sided bootstrap p-value.
    merged : pd.DataFrame
        Aligned data used for comparison.
    """
    df_hc = df_hc[~df_hc[value_col].isna()]
    df_pd = df_pd[~df_pd[value_col].isna()]
    # Determine join keys automatically
    join_keys = ["region"]
    if "hemisphere" in df_hc.columns and "hemisphere" in df_pd.columns:
        join_keys.append("hemisphere")

    merged = pd.merge(
        df_hc[join_keys + [value_col]],
        df_pd[join_keys + [value_col]],
        on=join_keys,
        suffixes=("_HC", "_PD"),
        how="inner",
    )

    if merged.empty:
        raise ValueError("No overlapping regions found between HC and PD datasets.")

    # Extract values
    hc_vals = merged[f"{value_col}_HC"].to_numpy()
    pd_vals = merged[f"{value_col}_PD"].to_numpy()

    # Observed difference
    observed = pd_vals.mean() - hc_vals.mean()

    # Use scipy's bootstrap
    res = bootstrap(
        (pd_vals, hc_vals),
        statistic=lambda a, b: a.mean() - b.mean(),
        paired=True,
        vectorized=False,
        n_resamples=n_boot,
        random_state=seed,
        confidence_level=ci,
        method="percentile",
    )

    ci_low, ci_high = res.confidence_interval.low, res.confidence_interval.high

    # Two-sided p-value from bootstrap distribution
    diffs = res.bootstrap_distribution
    p_value = (np.sum(np.abs(diffs) >= np.abs(observed)) + 1) / (len(diffs) + 1)

    # Permutation test as an alternative (uncomment if needed)
    def diff_mean(x, y):
        return y.mean() - x.mean()
    perm_res = permutation_test(
        (hc_vals, pd_vals),
        statistic=diff_mean,
        vectorized=False,
        n_resamples=n_boot,
        alternative="two-sided",
        random_state=seed,
    )
    p_value = perm_res.pvalue
    print("")
    print(f"Permutation test. observed difference: {perm_res.statistic:.4f}, p-value: {p_value:.4f}")

    return observed, (ci_low, ci_high), p_value, merged



In [113]:
df_boostrap = pd.DataFrame(columns=['metric', 'observed','95% CI', 'p-value', 'study'])

### Cortical Area

#### Cross-sectional at baseline

In [114]:
hc_baseline = pd.read_csv(input_dir / "npv_hc_baseline_area.csv")
pd_baseline = pd.read_csv(input_dir / "npv_pd_baseline_area.csv")
obs, ci, p, merged = bootstrap_group_diff(hc_baseline, pd_baseline)
df_boostrap.loc[len(df_boostrap)] = {'metric':'area','observed':obs, '95% CI':ci, 'p-value':p, 'study':'cross-sectional'}
print(f"Δ(PD-HC) = {obs:.3f}, p = {p:.3f}, 95% CI = [{ci[0]:.3f}, {ci[1]:.3f}]")
if p < alpha:
  print(f"Null-hypothesis rejected at alpha={alpha}")


Permutation test. observed difference: 0.0053, p-value: 0.7339
Δ(PD-HC) = 0.005, p = 0.734, 95% CI = [-0.002, 0.013]


#### Longitudinal

In [115]:
hc_baseline = pd.read_csv(input_dir / "npv_hc_area_longitudinal.csv")
pd_baseline = pd.read_csv(input_dir / "npv_pd_area_longitudinal.csv")
obs, ci, p, merged = bootstrap_group_diff(hc_baseline, pd_baseline)
df_boostrap.loc[len(df_boostrap)] = {
    "metric": "area",
    "observed": obs,
    "95% CI": ci,
    "p-value": p,
    "study": "longitudinal",
}
print(f"Δ(PD-HC) = {obs:.3f}, p = {p:.3f}, 95% CI = [{ci[0]:.3f}, {ci[1]:.3f}]")
if p < alpha:
    print(f"Null-hypothesis rejected at alpha={alpha}")


Permutation test. observed difference: 0.0207, p-value: 0.4818
Δ(PD-HC) = 0.021, p = 0.482, 95% CI = [-0.002, 0.044]


### Cortical Thickness

#### Cross-sectional at baseline

In [116]:
hc_baseline = pd.read_csv(input_dir / "npv_hc_baseline_thickness.csv")
pd_baseline = pd.read_csv(input_dir / "npv_pd_baseline_thickness.csv")
obs, ci, p, merged = bootstrap_group_diff(hc_baseline, pd_baseline)
df_boostrap.loc[len(df_boostrap)] = {
    "metric" : "thickness",
    "observed": obs,
    "95% CI": ci,
    "p-value": p,
    "study": "cross-sectional",
}
print(f"Δ(PD-HC) = {obs:.3f}, p = {p:.3f}, 95% CI = [{ci[0]:.3f}, {ci[1]:.3f}]")
if p < alpha:
  print(f"Null-hypothesis rejected at alpha={alpha}")


Permutation test. observed difference: 0.0265, p-value: 0.0326
Δ(PD-HC) = 0.027, p = 0.033, 95% CI = [0.020, 0.034]
Null-hypothesis rejected at alpha=0.05


#### Longitudinal

In [117]:
hc_baseline = pd.read_csv(input_dir / "npv_hc_thickness_longitudinal.csv")
pd_baseline = pd.read_csv(input_dir / "npv_pd_thickness_longitudinal.csv")
obs, ci, p, merged = bootstrap_group_diff(hc_baseline, pd_baseline)
df_boostrap.loc[len(df_boostrap)] = {
    "metric": "thickness",
    "observed": obs,
    "95% CI": ci,
    "p-value": p,
    "study": "longitudinal",
}
print(f"Δ(PD-HC) = {obs:.3f}, p = {p:.3f}, 95% CI = [{ci[0]:.3f}, {ci[1]:.3f}]")
if p < alpha:
    print(f"Null-hypothesis rejected at alpha={alpha}")


Permutation test. observed difference: 0.0023, p-value: 0.9187
Δ(PD-HC) = 0.002, p = 0.919, 95% CI = [-0.014, 0.020]


### Cortical Volume

#### Cross-sectional at baseline

In [118]:
hc_baseline = pd.read_csv(input_dir / "npv_hc_baseline_volume.csv")
pd_baseline = pd.read_csv(input_dir / "npv_pd_baseline_volume.csv")
obs, ci, p, merged = bootstrap_group_diff(hc_baseline, pd_baseline)
df_boostrap.loc[len(df_boostrap)] = {
  "metric": "volume",
    "observed": obs,
    "95% CI": ci,
    "p-value": p,
    "study": "cross-sectional",
}
print(f"Δ(PD-HC) = {obs:.3f}, p = {p:.3f}, 95% CI = [{ci[0]:.3f}, {ci[1]:.3f}]")
if p < alpha:
  print(f"Null-hypothesis rejected at alpha={alpha}")


Permutation test. observed difference: 0.0126, p-value: 0.3706
Δ(PD-HC) = 0.013, p = 0.371, 95% CI = [0.004, 0.022]


#### Longitudinal

In [119]:
hc_baseline = pd.read_csv(input_dir / "npv_hc_volume_longitudinal.csv")
pd_baseline = pd.read_csv(input_dir / "npv_pd_volume_longitudinal.csv")
obs, ci, p, merged = bootstrap_group_diff(hc_baseline, pd_baseline)
df_boostrap.loc[len(df_boostrap)] = {
    "metric": "volume",
    "observed": obs,
    "95% CI": ci,
    "p-value": p,
    "study": "longitudinal",
}
print(f"Δ(PD-HC) = {obs:.3f}, p = {p:.3f}, 95% CI = [{ci[0]:.3f}, {ci[1]:.3f}]")
if p < alpha:
    print(f"Null-hypothesis rejected at alpha={alpha}")


Permutation test. observed difference: 0.0246, p-value: 0.3698
Δ(PD-HC) = 0.025, p = 0.370, 95% CI = [0.002, 0.049]


### Subcortical Volume

#### Cross-sectional at baseline

In [120]:
hc_baseline = pd.read_csv(input_dir / "npv_hc_baseline_subcortical_volume.csv")
pd_baseline = pd.read_csv(input_dir / "npv_pd_baseline_subcortical_volume.csv")
obs, ci, p, merged = bootstrap_group_diff(hc_baseline, pd_baseline)
df_boostrap.loc[len(df_boostrap)] = {
    "metric": "subcortical_volume",
    "observed": obs,
    "95% CI": ci,
    "p-value": p,
    "study": "cross-sectional",
}
print(f"Δ(PD-HC) = {obs:.3f}, p = {p:.3f}, 95% CI = [{ci[0]:.3f}, {ci[1]:.3f}]")
if p < alpha:
  print(f"Null-hypothesis rejected at alpha={alpha}")


Permutation test. observed difference: 0.0141, p-value: 0.6457
Δ(PD-HC) = 0.014, p = 0.646, 95% CI = [0.004, 0.025]


#### Longitudinal

In [121]:
hc_baseline = pd.read_csv(input_dir / "npv_hc_subcortical_volume_longitudinal.csv")
pd_baseline = pd.read_csv(input_dir / "npv_pd_subcortical_volume_longitudinal.csv")
obs, ci, p, merged = bootstrap_group_diff(hc_baseline, pd_baseline)
df_boostrap.loc[len(df_boostrap)] = {
    "metric": "subcortical_volume",
    "observed": obs,
    "95% CI": ci,
    "p-value": p,
    "study": "longitudinal",
}
print(f"Δ(PD-HC) = {obs:.3f}, p = {p:.3f}, 95% CI = [{ci[0]:.3f}, {ci[1]:.3f}]")
if p < alpha:
    print(f"Null-hypothesis rejected at alpha={alpha}")


Permutation test. observed difference: -0.0267, p-value: 0.4894
Δ(PD-HC) = -0.027, p = 0.489, 95% CI = [-0.047, -0.006]


In [122]:
from rich.console import Console
from rich.table import Table
from rich.text import Text
from rich import box


def show_bootstrap_table(df):
    console = Console()

    table = Table(
        title="Bootstrap Comparison of NAVR Between HC and PD",
        title_style="bold white",
        header_style="bold white",
        box=box.SQUARE_DOUBLE_HEAD,
        border_style="white",
        show_lines=False,
        title_justify="center",
    )

    # Columns
    table.add_column("Metric", justify="left", style="bold cyan")
    table.add_column("Observed Δ", justify="right", style="bold white")
    table.add_column("95% CI", justify="center", style="yellow")
    table.add_column("p-value", justify="right", style="white")
    table.add_column("Study", justify="center", style="bold magenta")

    for _, row in df.iterrows():
        metric = row.name if "metric" not in df.columns else row["metric"]

        # format observed
        obs = f"{row['observed']:.3f}"

        # parse confidence interval
        ci = row["95% CI"]
        if isinstance(ci, tuple) or hasattr(ci, "__iter__"):
            ci_text = f"[{ci[0]:.3f}, {ci[1]:.3f}]"
        else:
            # if stored as np.float64 tuple string
            low, high = eval(str(ci))
            ci_text = f"[{float(low):.3f}, {float(high):.3f}]"

        # format p-value color
        pval = float(row["p-value"])
        if pval < 0.001:
            p_style = "bold bright_red"
        elif pval < 0.01:
            p_style = "red"
        elif pval < 0.05:
            p_style = "yellow"
        else:
            p_style = "dim white"

        p_text = Text(f"{pval:.3f}", style=p_style)

        # add row
        table.add_row(str(metric), f"{obs}", ci_text, p_text, row.get("study", ""))

    console.print(table)

show_bootstrap_table(df_boostrap)

                  Bootstrap Comparison of NAVR Between HC and PD                  
┌────────────────────┬────────────┬──────────────────┬─────────┬─────────────────┐
│ Metric             │ Observed Δ │      95% CI      │ p-value │      Study      │
╞════════════════════╪════════════╪══════════════════╪═════════╪═════════════════╡
│ area               │      0.005 │ [-0.002, 0.013]  │   0.734 │ cross-sectional │
│ area               │      0.021 │ [-0.002, 0.044]  │   0.482 │  longitudinal   │
│ thickness          │      0.027 │  [0.020, 0.034]  │   0.033 │ cross-sectional │
│ thickness          │      0.002 │ [-0.014, 0.020]  │   0.919 │  longitudinal   │
│ volume             │      0.013 │  [0.004, 0.022]  │   0.371 │ cross-sectional │
│ volume             │      0.025 │  [0.002, 0.049]  │   0.370 │  longitudinal   │
│ subcortical_volume │      0.014 │  [0.004, 0.025]  │   0.646 │ cross-sectional │
│ subcortical_volume │     -0.027 │ [-0.047, -0.006] │   0.489 │  longitudinal   │
└────────────────────┴────────────┴──────────────────┴─────────┴─────────────────┘